# Compare multiple string columns to return similarity scores

In [ ]:
import numpy as np
import pandas as pd
from difflib import SequenceMatcher

In [ ]:
file = 'data1.csv'
data1 = pd.read_csv(file)

In [ ]:
file = 'data2.csv'
data2 = pd.read_csv(file)

In [ ]:
frames = [data1, data2]
 
res_dt = pd.concat(frames).reset_index()
del res_dt['index']

In [ ]:
data = res_dt.replace(np.nan, '', regex=True)

In [ ]:
data.drop_duplicates(subset='name', keep='first', inplace=True)

Use SequenceMatcher and find string similarity between string columns, score is out of 100, round up (2 digits)

In [ ]:
def sim_metric(df, col1, col2):
    return round((SequenceMatcher(None, df[col1], df[col2]).ratio()*100),2)

Compute similarity score for multiple string columns, compare and return maximum similarity score

In [ ]:
data['score'] = data.apply(sim_metric,
                          args=('col1', 'col2'), axis=1)
data['score2'] = data.apply(sim_metric,
                          args=('col1', 'col3'), axis=1)

data['max_score'] = data[['score1', 'score2']].max(axis=1)
data

In [ ]:
data.to_csv("test_score.csv")